# Single Process

In [11]:
import jaydebeapi
import os
import pandas as pd
import time

user='Nsadghi'
password ='good4gap'
mypath='C:/Users/nsadghi/Desktop/JavaDrivers'

import sys
sys.path.append(mypath)


from os import listdir
from os.path import isfile, join

try:
    conn_teradata = jaydebeapi.connect(jclassname="com.teradata.jdbc.TeraDriver",
                                               url="jdbc:teradata://10.101.83.123/LOGMECH=TD2",
                                               driver_args=[user, password],
                                               jars=[mypath+'/tdgssconfig.jar', mypath+'/terajdbc4.jar'])
    if conn_teradata.cursor():
        print 'Connection has been made succesfully!'

except:
    print 'Failed!'

Connection has been made succesfully!


In [ ]:
start_time = time.time()
DF = pd.read_sql_query('''<my_query>''', conn_teradata)
print("--- %s seconds ---" % (time.time() - start_time))

# Multi Process

In [ ]:
import sys
import multiprocessing as mp
import time
import pandas as pd
from functools import partial
from tqdm import tqdm

from QueryWrapper import QueryWrapper
from teradata_utils import *

if __name__ == "__main__":
    start_time = time.time()

    query_raw = QueryWrapper(mypath,'nima.txt')
    
    inputs = (8,1,1,'2016-12-01','2017-07-31',710,42)
    
    start_date_index = 3
    end_date_index = 4
    
    
    dates = list(date_range(inputs[start_date_index], inputs[end_date_index], mp.cpu_count()))
    
    list_inputs=[]
    for d in dates:
        lstinputs = list(inputs)
        lstinputs[start_date_index] = d[0]
        lstinputs[end_date_index] = d[1]
        inputs = tuple(lstinputs)
        list_inputs.append(inputs)
        
    pool = mp.Pool(processes = mp.cpu_count())  
    dfs=[]
    for df in tqdm(pool.imap_unordered(partial(read_chunks, query=query_raw, user='Nsadghi'),list_inputs), total=mp.cpu_count()):
        dfs.append(df)
        
    pool.close()  
    pool.join() 
    
    dfs = pd.concat(dfs, axis=0)
    print("--- %s seconds ---" % (time.time() - start_time))